In [178]:
import random
random.seed(109)

In [257]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):

        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        player1pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player2pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        if (pit > self.pits_per_player or pit <= 0): return False
        
        if (self.current_player == 1):
            if player1pits[pit-1] > 0:
                #print("Player 1 chose pit: ", pit)
                return True
            else:
                return False
        else:
            if player2pits[pit-1] > 0:
                #print("Player 2 chose pit: ", pit)
                return True
            else:
                return False

        pass
        
    def random_move_generator(self):

        move = random.randint(1, self.pits_per_player)
        while not self.valid_move(move): move = random.randint(1, self.pits_per_player)
    

        return move
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        
        - On every turn, select a pit and distribute its stones in a counter-clockwise direction.
        - If the last stone lands in the player's mancala, in an opponent's pit, or in one of the player's non-empty pits,
            no further action is taken, and the current player's turn ends.
        - If the last stone lands in the current player's empty pit and the opposite pit on the opponent's side 
            has some stones, collect all those stones, including the one that just landed, and place them into the 
            current player's mancala.
        """
        pitsPerPlayer = self.pits_per_player
        n = len(self.board)
        while(not self.winning_eval()):
            ## Get the number of stones in the pit
            pit = self.random_move_generator()
            stones = self.board[pit - 1]
            self.board[pit-1] = 0
            for i in range(stones):
                ## If pit is smaller than 
                ## And i is final stone
                ## And pit has 0 stones
                if(((pit+i) %n) < pitsPerPlayer and i == (stones - 1) and self.board[(pit+i) %n] == 0 and self.board[n - pit - i - 2] != 0):
                    #print("Index: ", n - pit - i - 2)
                    self.board[pitsPerPlayer] += 1 + self.board[n - pit - i - 2]
                    self.board[n - pit - i - 2] = 0
                else:
                    self.board[(pit+i) % n] += 1

            self.moves.append((self.current_player, pit))
            self.current_player = 2
            #self.display_board()
            if(self.winning_eval()):
                break
            move = self.random_move_generator()
            pits2 = move + pitsPerPlayer + 1
            stones = self.board[pits2 - 1]
            self.board[pits2-1] = 0
            for i in range(stones):
                ## Is smaller than len - 1 
                ## Is bigger than pits per player
                if((pits2+i)%n < (n-1) and (pits2+i)%n > pitsPerPlayer and i == (stones - 1)
                   and self.board[(pits2+i) %n] ==0 and  self.board[n - pits2 - i - 2] != 0):
                    #print("Index: ", n - pits2 - i - 2)
                    self.board[n - 1] += 1 + self.board[n - pits2 - i - 2]
                    self.board[n - pits2 - i - 2] = 0
                else:
                    self.board[(pits2+i) % (n)] += 1

            self.moves.append((self.current_player, pit))
            self.current_player = 1
            #self.display_board()
        #print("GameOver")    
            
        
        
 
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        player1pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player2pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        # Reset count
        count1 = 0
        count2 = 0
        for i in player2pits:
            if (i == 0):
                count1 +=1
        
    
        # Reset count
        for i in player1pits:
            if (i == 0):
                count2 += 1
                
        if (count1 == self.pits_per_player or count2 == self.pits_per_player): return True
        else: return False

        pass


In [292]:
game = Mancala(6, 3)


In [293]:
game.play(3)
game.display_board()

P1               P2
     ____9____     
1 -> | 4 | 0 | <- 6
2 -> | 1 | 0 | <- 5
3 -> | 2 | 0 | <- 4
4 -> | 0 | 0 | <- 3
5 -> | 1 | 0 | <- 2
6 -> |_2_|_0_| <- 1
         17         
Turn: P1


In [294]:
## Play 100 games
import numpy as np
arr = [0, 0, 0]
totMoves = []
for i in range(100):
    #Start new game
    game = Mancala(6, 3)
    game.play(0)
    totMoves.append(len(game.moves))
    #game.display_board()
    # If player 1 has more stones
    if (game.board[3] > game.board[7]):
        arr[0] += 1
    # If player 2 has more stones
    elif (game.board[3] < game.board[7]):
        arr[1] += 1
    # If tie
    else:
        arr[2] += 1
arr

[25, 28, 47]

### Random player win rates

In [305]:
print("Random player 1 has a winrate of:", arr[0], "%")
print("Random player 2 has a winrate of:", arr[1], "%")
print("The rate of ties is:", arr[2], "%")

Random player 1 has a winrate of: 25 %
Random player 2 has a winrate of: 28 %
The rate of ties is: 47 %


### Average number of moves for game to end

In [308]:
totMoves = np.array(totMoves)
print("The average number of moves for game to end is:", np.average(totMoves))

The average number of moves for game to end is: 32.96
